In [5]:
import os
import sys
# 특정 디렉토리 아래의 모든 파일의 절대경로 목록을 리턴하는 함수
def allfiles(path):
    res = []
    for root, dirs, files in os.walk(path):
        rootpath = os.path.join(os.path.abspath(path), root)

        for file in files:
            filepath = os.path.join(rootpath, file)
            if 'xlsx' in filepath:
                res.append(filepath)
    return res

import re
def ex_keyword(h_x,keyword):
    for i in range(len(h_x)):
        try:
            h_x[i] = h_x[i].replace(keyword,"")
        except:
            h_x[i] = str(h_x[i])
            pass
    return h_x

# freq_1: 첫번째
def freq_1(doc_all):
    from sklearn.feature_extraction.text import CountVectorizer
    from konlpy.tag import Twitter
    twit = Twitter()
    def tokenizer2 (doc):
        tagged = ['/'.join(t) for t in twit.pos(doc)]
        result_words = [word for word in tagged if word.split('/')[-1] in ['Noun','Adjective']]
        result_nouns = [word.split('/')[0] for word in result_words if len(word.split('/')[0]) > 1]
        return result_nouns
    count_vect = CountVectorizer(tokenizer=tokenizer2,min_df=0)
    count_all = count_vect.fit_transform(doc_all)
    from sklearn.feature_extraction.text import TfidfTransformer
    tf_transformer = TfidfTransformer().fit(count_all)
    tf_all = tf_transformer.transform(count_all)
    tf_all_array = tf_all.toarray()
    sort_index = tf_all_array.argsort(axis=1)
    count_dict = count_vect.vocabulary_
    subset_a = []
    for index in sort_index[0][-20:]:
        important_word = index
        key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
        subset_a.append(key)
    subset_b = []
    for index in sort_index[1][-20:]:
        important_word = index
        key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
        subset_b.append(key)
    subset_c = []
    for index in sort_index[2][-20:]:
        important_word = index
        key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
        subset_c.append(key)
    result = [subset_a,subset_b,subset_c]
    return result

# freq: 두번째
def freq(level, subset):
    from sklearn.feature_extraction.text import CountVectorizer
    from konlpy.tag import Twitter
    twit = Twitter()
    def tokenizer2 (doc):
        tagged = ['/'.join(t) for t in twit.pos(doc)]
        result_words = [word for word in tagged if word.split('/')[-1] in ['Noun','Adjective']]
        result_nouns = [word.split('/')[0] for word in result_words if len(word.split('/')[0]) > 1]
        return result_nouns
    count_vect = CountVectorizer(tokenizer=tokenizer2,min_df=0)
    count_all = count_vect.fit_transform(level)
    count_sub = count_vect.transform(subset)
    from sklearn.feature_extraction.text import TfidfTransformer
    tf_transformer = TfidfTransformer().fit(count_all)
    tf_all = tf_transformer.transform(count_sub)
    tf_all_array = tf_all.toarray()
    sort_index = tf_all_array.argsort(axis=1)
    count_dict = count_vect.vocabulary_
    subset_a = []
    for index in sort_index[0][-20:]:
        important_word = index
        key = list(count_dict.keys())[list(count_dict.values()).index(important_word)]
        subset_a.append(key)
    return subset_a

def main(doc_all,level_all):
    step_1 = freq_1(doc_all)
    x = 0
    box_all = []
    for i in step_1: # 0: pos, 1: neu, 2: neg
        box_all.append([])
        for j in range(len(i)): # extract keyword: j
            box_all[x].append([])
            for k in level_all[x]:
                if (i[j] in k) and (len(k)>len(i[j])):
                    box_all[x][j].append(k)
        x += 1
    z = 0
    rebox_all = []
    for i in box_all:
        rebox_all.append([])
        for j in range(len(i)):
            rebox_all[z].append(freq(level_all[z],i[j]))
        z += 1
    return rebox_all

# file 열기

import pandas as pd
import os
curDir = os.getcwd()
# path에 파일명 입력
# example
# path = curDir+'/raw_data/twitter_notadv.txt'

######################################
############# INPUT HERE #############
######################################

path = '/Users/EternalFlow/Desktop/indatalab/nlp/projects/wannaone(new)/result_data/twit_170811_wannaone'
files = allfiles(path)

# 단일 파일의 경우 for 문을 제거하고 path에 절대경로 입력
# 마지막에 writer 경로에 새폴더를 넣을 경우 미리 폴더를 만들어주어야 함
for i in files:
    path = i
    original_file = path[path.rfind('/')+1:path.rfind('.')]
    #input_data=pd.read_table(path,delimiter='\t',encoding='utf-8',header=None)
    input_data=pd.read_excel(path,encoding='utf-8')
    input_data = input_data.dropna(axis=0)
    contents = input_data['contents'].as_matrix()

    # channel = input_data['channel'].as_matrix()
    # date = input_data['date'].as_matrix()
    # contents = input_data['contents'].as_matrix()
    point = input_data['point'].as_matrix()
    # label = input_data['predict_label'].as_matrix()
    # sub_contents = ex_keyword(contents,'워너원')
    # point: 50 ~ 100
    level_a = []
    for i in range(len(contents)):
        if point[i] >= 50:
            level_a.append(str(contents[i]))

    # point: -50 ~ 50
    level_b = []
    for i in range(len(contents)):
        if point[i] >= -50 and point[i] < 50 :
            level_b.append(str(contents[i]))

    # point: -100 ~ -50
    level_c = []
    for i in range(len(contents)):
        if point[i] < -50:
            level_c.append(str(contents[i]))
    doc_a = ''.join(level_a)
    doc_b = ''.join(level_b)
    doc_c = ''.join(level_c)
    level_all = [level_a,level_b,level_c]
    doc_all = [doc_a,doc_b,doc_c]

    result_set = freq_1(doc_all)
    test = main(doc_all,level_all)
    subset_a = pd.DataFrame(data=test[0],index=result_set[0])
    subset_b = pd.DataFrame(data=test[1],index=result_set[1])
    subset_c = pd.DataFrame(data=test[2],index=result_set[2])
    writer = pd.ExcelWriter(curDir+'/word_data/twit_170811_recap/word_'+original_file+'.xlsx') #here
    subset_a.to_excel(writer,'Positve')
    subset_b.to_excel(writer,'Neutral')
    subset_c.to_excel(writer,'Negative')
    writer.save()
    print(original_file)
    '''
    except:
        print(path)
        count_pos = len(input_data.loc[input_data[5] == 'POS'])
        count_neu = len(input_data.loc[input_data[5] == 'NEU'])
        count_neg = len(input_data.loc[input_data[5] == 'NEG'])
        count_all = len(input_data)
        counts = [count_pos,count_neu,count_neg]
        print("긍정: {}".format(count_pos), "비중: {:.2f}".format(count_pos/count_all))
        print("중립: {}".format(count_neu), "비중: {:.2f}".format(count_neu/count_all))
        print("부정: {}".format(count_neg), "비중: {:.2f}".format(count_neg/count_all))
        print(i)
        pass
    '''

predicted_170811_wannaone_bae_jinyoung_Woo_merged
predicted_170811_wannaone_ha_sungwoon_Woo_merged
predicted_170811_wannaone_hwang_minhyun_Woo_merged
predicted_170811_wannaone_kang_daniel_Woo_merged
predicted_170811_wannaone_kim_jaehwan_Woo_merged
predicted_170811_wannaone_lai_kuanlin_Woo_merged
predicted_170811_wannaone_lee_daehwi_Woo_merged
predicted_170811_wannaone_ong_sungwoo_Woo_merged
predicted_170811_wannaone_park_jihoon_Woo_merged
predicted_170811_wannaone_park_woojin_Woo_merged
predicted_170811_wannaone_yoon_jisung_Woo_merged
